# 7 機械学習の前処理を行う 10本

## 7.1 機械学習で予測するデータを設定

In [1]:
import seaborn as sns
dataset = sns.load_dataset('titanic')
display(dataset)
label = dataset.pop('survived')
display(label)

,survived,pclass,sex,age,sibsp,parch,fare,embarked,class,who,adult_male,deck,embark_town,alive,alone
0,0,3,male,22.0,1,0,7.2500,S,Third,man,True,NaN,Southampton,no,False
1,1,1,female,38.0,1,0,71.2833,C,First,woman,False,C,Cherbourg,yes,False
2,1,3,female,26.0,0,0,7.9250,S,Third,woman,False,NaN,Southampton,yes,True
3,1,1,female,35.0,1,0,53.1000,S,First,woman,False,C,Southampton,yes,False
4,0,3,male,35.0,0,0,8.0500,S,Third,man,True,NaN,Southampton,no,True
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
886,0,2,male,27.0,0,0,13.0000,S,Second,man,True,NaN,Southampton,no,True
887,1,1,female,19.0,0,0,30.0000,S,First,woman,False,B,Southampton,yes,True
888,0,3,female,NaN,1,2,23.4500,S,Third,woman,False,NaN,Southampton,no,False
889,1,1,male,26.0,0,0,30.0000,C,First,man,True,C,Cherbourg,yes,True


0      0
1      1
2      1
3      1
4      0
      ..
886    0
887    1
888    0
889    1
890    0
Name: survived, Length: 891, dtype: int64

## 7.2 Train/Test データの分割

Train/Testデータを分けることで、modelsの精度を確認することができる。
Training Dataで精度を確認してしまうと、Training Dataに対して過学習しているかなどを確認することができない。そこでテスト用のデータを用意することでより正確に精度の検証を実行することが可能

- テストデータを分割する方法
  - 2分割 (Train/Test)
  - 3分割 (Train/Test/Validation)
  - 5分割

などの手法が存在する

In [2]:
from sklearn.model_selection import train_test_split
# 分割比率: default= 7:3 (test_size=0.2などと指定することで比率を変更することが可能)
# stratify: 均等に分割したいデータを指定している
train_ds, test_ds, train_label, test_label = train_test_split(dataset, label, test_size=0.2, random_state=0, stratify=label)
print(train_ds.shape)
print(test_ds.shape)

(712, 14)
(179, 14)


## 7.3 データを機械学習に適した形式へ変換

- 機械学習に使用すべきでない変数
  - 目的変数と直接的に関係のあるデータ
    - 目的変数と直接的に関係のあるデータは必ず除去すること 
    - **e.g.: alive column** これは現在生きているかどうかという情報であるため、生還したかどうかが前提になっている
  - 長さ(cm)、長さ(m)のように説明変数でほぼ同じデータである変数
    - 未知のデータを予測する際に知り得ない情報や直接的に関係のある情報は使用できない(データリーク)
    - **e.g.: embark_town column** embarkedと同じ事柄を表すデータなので使用しないこと

In [5]:
train_ds.drop(columns=['alive', 'embark_town'])
train_ds.head(5)

,pclass,sex,age,sibsp,parch,fare,embarked,class,who,adult_male,deck,embark_town,alive,alone
502,3,female,NaN,0,0,7.6292,Q,Third,woman,False,NaN,Queenstown,no,True
464,3,male,NaN,0,0,8.0500,S,Third,man,True,NaN,Southampton,no,True
198,3,female,NaN,0,0,7.7500,Q,Third,woman,False,NaN,Queenstown,yes,True
765,1,female,51.0,1,0,77.9583,S,First,woman,False,D,Southampton,yes,False
421,3,male,21.0,0,0,7.7333,Q,Third,man,True,NaN,Queenstown,no,True


encode categorical data by one-hot

`pd.get_dummies`により、文字列データを横に変換することが可能

1columnのみ1(=true)となり、それぞれの行の特徴を表す

In [6]:
import pandas as pd
one_hot_encoded = pd.get_dummies(train_ds)
one_hot_encoded.head(5)

,pclass,age,sibsp,parch,fare,adult_male,alone,sex_female,sex_male,embarked_C,...,deck_C,deck_D,deck_E,deck_F,deck_G,embark_town_Cherbourg,embark_town_Queenstown,embark_town_Southampton,alive_no,alive_yes
502,3,NaN,0,0,7.6292,False,True,1,0,0,...,0,0,0,0,0,0,1,0,1,0
464,3,NaN,0,0,8.0500,True,True,0,1,0,...,0,0,0,0,0,0,0,1,1,0
198,3,NaN,0,0,7.7500,False,True,1,0,0,...,0,0,0,0,0,0,1,0,0,1
765,1,51.0,1,0,77.9583,False,False,1,0,0,...,0,1,0,0,0,0,0,1,0,1
421,3,21.0,0,0,7.7333,True,True,0,1,0,...,0,0,0,0,0,0,1,0,1,0


encode categorical data by label encoding



In [7]:
from sklearn.preprocessing import LabelEncoder
label_encoded = train_ds.copy()
class_encoder = LabelEncoder()
label_encoded['class'] = class_encoder.fit_transform(label_encoded['class'])
label_encoded.head(5)

,pclass,sex,age,sibsp,parch,fare,embarked,class,who,adult_male,deck,embark_town,alive,alone
502,3,female,NaN,0,0,7.6292,Q,2,woman,False,NaN,Queenstown,no,True
464,3,male,NaN,0,0,8.0500,S,2,man,True,NaN,Southampton,no,True
198,3,female,NaN,0,0,7.7500,Q,2,woman,False,NaN,Queenstown,yes,True
765,1,female,51.0,1,0,77.9583,S,0,woman,False,D,Southampton,yes,False
421,3,male,21.0,0,0,7.7333,Q,2,man,True,NaN,Queenstown,no,True
